**Importation des Bibliothèques**

In [1]:
import numpy as np
import pandas as pd
import math
import os.path
from os import walk
import ntpath
from numpy import array
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

**Résumé**

Les fonctions de IFM:

ICL prend la température, l'humidité relative, la vitesse du vent, la pluie et une valeur ICL précédente pour produire la valeur ICL actuelle
   - ICL(17,42,25,0,85) = 87.692980092774448

IH prend la température, l'humidité relative, les précipitations, la valeur IH précédente, la latitude et le mois en cours pour produire la valeur IH actuelle
    - IH(17,42,0,6,45.98,4) = 8.5450511359999997

IS prend la température, la pluie, la valeur IS précédente, la latitude et le mois en cours pour produire la valeur IS actuelle
   - IS(17,0,15,45.98,4) = 19.013999999999999

IPI prend la vitesse du vent et la valeur ICL actuelle pour produire la valeur IPI actuelle
   - IPI(25,87.692980092774448) = 10.853661073655068

ICD prend les valeurs IH et IS actuelles pour produire la valeur ICD actuelle
   - ICD(8.5450511359999997,19.013999999999999) = 8.4904265358371838

IFM prend les valeurs IPI et ICD actuelles pour produire la valeur IFM actuelle
   - IFM(10.853661073655068,8.4904265358371838) = 10.096371392382368

calcIFM - cette fonction renvoie la valeur IFM actuelle en fonction de toutes les valeurs d'entrée : mois, température, humidité relative, vitesse du vent, pluie, ICL précédent, IH, IS et latitude
   - calcIFM(4,17,42,25,0,85,6,15,45.98) = 10.096371392382368

In [2]:
class InvalidLatitude(Exception):
    """Exception pour gérer les variables non couvertes par DataDict"""
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return repr(self.value) + " n'est pas une latitude valide."

**Calcul de l'Indice du Combustible Leger**

In [3]:
def ICL(T,H,FF,RR,ICLPrev):
    '''Calcule le code d'humidité du carburant fin d'aujourd'hui
    PARAMETRES
    ----------
    * T est la température de 12:00 LST en degrés Celsius
    * H est l'humidité relative de 12:00 LST en %
    * FF est la vitesse du vent à 12:00 LST en km/h
    * RR est la pluie cumulée sur 24 heures en mm, calculée à 12:00 LST
    * m0 est une valeur indiquée dans la base de données'''
    
    H = min(100.0,H)
    mo = 147.2 * (101.0 - ICLPrev) / (59.5 + ICLPrev)
    if RR > 0.5:
        rf = RR - 0.5

        if mo <= 150.0: 
            mr = mo + \
                 42.5 * rf * math.exp(-100.0 / (250.0 - mo)) * (1.0-math.exp(-6.93 / rf))
        else:

            mr = mo + \
                 42.5 * rf * math.exp(-100.0 / (251.0 - mo)) * (1.0-math.exp(-6.93 / rf)) + \
                 0.0015 * pow(mo - 150.0, 2) * pow(rf, 0.5)

        if mr > 250.0:
            mr = 250.0

        mo=mr
        
    ed = 0.942 * pow(H, 0.679) + \
         11.0 * math.exp((H - 100.0) / 10.0) + 0.18 * (21.1 - T) * (1.0 - math.exp(-0.115 * H))

    if mo > ed:
        ko = 0.424 * (1.0 - pow(H / 100.0, 1.7)) + \
             0.0694 * pow(FF, 0.5) * (1.0 - pow(H / 100.0, 8))

        kd = ko * 0.581 * math.exp(0.0365 * T)

        m = ed + (mo - ed) * pow(10.0,-kd)

    else:
        ew = 0.618 * pow(H,0.753) + \
             10.0 * math.exp((H - 100.0) / 10.0) + \
             0.18 * (21.1 - T) * (1.0 - math.exp(-0.115 * H))
        if mo < ew:
            k1 = 0.424 * (1.0 - pow((100.0 - H) / 100.0, 1.7)) + \
                 0.0694 * pow(FF, 0.5) * (1.0 - pow((100.0 - H) / 100.0, 8))

            kw = k1 * 0.581 * math.exp(0.0365 * T)


            m = ew - (ew - mo) * pow(10.0, -kw)
        else:
            m = mo
    return 59.9 * (250.0 - m) / (147.2 + m)

In [4]:
""" DayLength calcule la longueur du jour (le temps entre le lever du soleil et coucher du soleil) compte tenu du jour de l'année et de la latitude du lieu."""
def DayLength(MONTH):
    '''Estimation de la durée du jour en fonction du mois et de la latitude'''
    LeN= {'1': 6.5, '2': 7.5 , '3': 9.0, '4':12.8 , '5':13.9, '6':13.9, '7':12.4, '8':10.9, '9':9.4,'10':8.0, '11':7.0, '12':6.0}
    return LeN.get(str(MONTH))

In [5]:
""" DayLength46N = [ 6.5,  7.5,  9.0, 12.8, 13.9, 13.9, 12.4, 10.9,  9.4,  8.0,  7.0,  6.0]
    DayLength20N = [ 7.9,  8.4,  8.9,  9.5,  9.9, 10.2, 10.1,  9.7,  9.1,  8.6,  8.1,  7.8]
    DayLength20S = [10.1,  9.6,  9.1,  8.5,  8.1,  7.8,  7.9,  8.3,  8.9,  9.4,  9.9, 10.2]
    DayLength40S = [11.5, 10.5,  9.2,  7.9,  6.8,  6.2,  6.5,  7.4,  8.7, 10.0, 11.2, 11.8]"""

"""retVal = None

    if Latitude<= 90 and Latitude > 33:
        retVal = DayLength46N[MONTH-1]
    elif Latitude <= 33 and Latitude > 0.0:
        retVal = DayLength20N[MONTH-1]
    elif Latitude <= 0.0 and Latitude > -30.0:
        retVal = DayLength20S[MONTH-1]
    elif Latitude <= -30.0 and Latitude >= -90.0:
        retVal = DayLength40S[MONTH-1]

    if retVal==None:
        raise InvalidLatitude(Latitude)"""

'retVal = None\n\n    if Latitude<= 90 and Latitude > 33:\n        retVal = DayLength46N[MONTH-1]\n    elif Latitude <= 33 and Latitude > 0.0:\n        retVal = DayLength20N[MONTH-1]\n    elif Latitude <= 0.0 and Latitude > -30.0:\n        retVal = DayLength20S[MONTH-1]\n    elif Latitude <= -30.0 and Latitude >= -90.0:\n        retVal = DayLength40S[MONTH-1]\n\n    if retVal==None:\n        raise InvalidLatitude(Latitude)'

**Calcul de l'Indice de l'Humus**

In [6]:
def IH(T,H,RR,IHPrev,MONTH):
    ''' Calcule le code d'humidité Duff d'aujourd'hui
    PARAMÈTRES
    ----------
    * T est la température de 12:00 LST en degrés Celsius
    * H est l'humidité relative de 12:00 LST en %
    * RR est la pluviométrie cumulée sur 24 heures en mm, calculée à 12h00 LST
    * IHPrev est l'IH de la veille
    * Month est le mois de l'année (1..12) pour les calculs du jour en cours.'''

    H = min(100.0,H)
    mo = 20.0 + math.exp(5.6348 - IHPrev / 43.43)
    if RR > 1.5:
        re = 0.92 * RR - 1.27
        if IHPrev <= 33.0:
            b = 100.0 / (0.5 + 0.3 * IHPrev)
        else:
            if IHPrev <= 65.0:
                b = 14.0 - 1.3 * math.log(IHPrev)
            else:
                b = 6.2 * math.log(IHPrev) - 17.2
        
        mr = mo + 1000.0 * re / (48.77 + b * re)
        
        pr = 244.72 - 43.43 * math.log(mr - 20.0)

        if pr > 0.0:
            IHPrev = pr
        else:
            IHPrev = 0.0
    else:
        mr=mo
    if T > -1.1:
        d1 = DayLength(MONTH)
        k = 1.894 * (T + 1.1) * (100.0 - H) * d1 * 0.000001
    else:
        k = 0.0

    return IHPrev + 100.0 * k

In [7]:
def DryingFactor(Month):
    '''Estimation de la durée du jour en fonction du mois'''
    LfN= {'1': -1.6, '2': -1.6 , '3':-1.6, '4':0.9 , '5':3.8, '6':5.8, '7':6.4, '8':5.0, '9':2.4,'10':0.4, '11':-1.6, '12':-1.6}
    return LfN.get(str(Month))

In [8]:
"""LfN = [-1.6, -1.6, -1.6, 0.9, 3.8, 5.8, 6.4, 5.0, 2.4, 0.4, -1.6, -1.6]

LfS = [6.4, 5.0, 2.4, 0.4, -1.6, -1.6, -1.6, -1.6, -1.6, 0.9, 3.8, 5.8]

 if Latitude > 0:
        retVal = LfN[Month]
    elif Latitude <= 0.0:
        retVal = LfS[Month]

    return retVal"""

'LfN = [-1.6, -1.6, -1.6, 0.9, 3.8, 5.8, 6.4, 5.0, 2.4, 0.4, -1.6, -1.6]\n\nLfS = [6.4, 5.0, 2.4, 0.4, -1.6, -1.6, -1.6, -1.6, -1.6, 0.9, 3.8, 5.8]\n\n if Latitude > 0:\n        retVal = LfN[Month]\n    elif Latitude <= 0.0:\n        retVal = LfS[Month]\n\n    return retVal'

**Calcul de l'Indice de Sécheresse**

In [9]:
def IS(T,RR,ISPrev,Month):
    """Calcule les paramètres du code de sécheresse d'aujourd'hui
    PARAMÈTRES
    ----------
    T est la température de 12:00 LST en degrés Celsius
    RR est la pluviométrie cumulée sur 24 heures en mm, calculée à 12h00 LST
    IS0 est le IS initial
    LAT est la latitude en degrés décimaux de l'emplacement pour lequel les calculs sont effectués
    MONTH est le mois de l'année (1..12) pour les calculs du jour en cours"""   
    if RR > 2.8:
        rd = 0.83 * RR - 1.27
        Q0 = 800.0 * math.exp(-ISPrev / 400.0)
        Qr = Q0 + 3.94 * rd
        IS0 = 400.0 * math.log(800.0 / Qr)
    else:
        Qr = ISPrev
        IS0 = 400.0 * math.log(800.0 / Qr)
        
    Lf = DryingFactor(Month) #a verifier que month-1 dans DryingFactor
    if T > -2.8:
        V = 0.36 * (T+2.8) + Lf
        IS0= 0.0
    else:
        V = Lf
        if V < 0.0:
            V = 0.0
        IS0=0.0
        
    S = IS0 + 0.5 * V

    return S

**Calcul de l'Indice de Propagation Initiale**

In [10]:
def IPI(FF,ICL):
    '''Calcule l'indice de propagation initial d'aujourd'hui
    PARAMÈTRES
    ----------
    FF est la vitesse du vent à 12:00 LST en km/h
    ICL est le FFMC du jour'''

    fF0 = math.exp(0.05039 * FF)

    m = 147.2 * (101.0 - ICL) / (59.5 + ICL)

    fF1 = 91.9 * math.exp(-0.1386 * m) * (1.0 + pow(m, 5.31) / 49300000.0)

    return 0.208 * fF0 * fF1 

**Calcul de l'Indice de Combustible Disponible**

In [11]:
def ICD(IH,IS):
    '''Calcule l'indice d'accumulation d'aujourd'hui
    PARAMÈTRES
    ----------
    IH est le code d'humidité Duff du jour en cours
    IS est le code de sécheresse du jour en cours'''

    if IH <= 0.4 * IS:
        U = 0.8 * IH * IS / (IH + 0.4 * IS)
    else:
        U = IH - (1.0 - 0.8 * IS / (IH + 0.4 * IS)) * \
            (0.92 + pow(0.0114 * IH, 1.7))

    return max(U,0.0)

**Calcul de l'Indice Foret Météo**

In [12]:
def IFM(IPI, ICD):
    '''Calcule l'indice météo d'incendie d'aujourd'hui
    PARAMÈTRES
    ----------
    IPI est l'IPI du jour en cours
    ICD est l'ICD du jour en cours'''

    if ICD <= 80.0:
        I = 0.626 * pow(ICD, 0.809) + 2.0
    else:
        I = 1000.0 / (25.0 + 108.64 * math.exp(-0.023 * ICD))

    return 0.1 * IPI * I

**Calcul de l'IFM final**

In [13]:
t1="C:/Users/Zaineb/Desktop/METEO/PFE/nouvelle bd 2021"
l1 = []
for f1 in walk(t1):
    l1.extend(f1)
L=l1[2]

for j in range(len(L)) :
    try:
        df1=pd.read_excel("C:/Users/Zaineb/Desktop/METEO/PFE/nouvelle bd 2021/" + L[j-1])
        t3="C:/Users/Zaineb/Desktop/METEO/PFE/OUTPUT"
        l3 = []
        for f3 in walk(t3):
            l3.extend(f3)
        O=l3[2]        
        df3= pd.read_excel("C:/Users/Zaineb/Desktop/METEO/PFE/OUTPUT/" + O[j-1])
        Date = L[j][6:-7]
        #df=df1.merge(df2, how= 'inner', on='STATION')
        tab=[]
        for i in range(len(df1)):
            stat= df1['STATION'][i]
            p1= ICL(df1['MAXI(J)'][i], df1['HRmax'][i],df1['FFmax'][i], df1['RRtot'][i], df3['ICL'][i])
            p2= IH(df1['MAXI(J)'][i], df1['HRmax'][i], df1['RRtot'][i], df3['IH'][i] , int(Date[4:-2]))
            p3= IS(df1['MAXI(J)'][i], df1['RRtot'][i], df3['IS'][i] , int(Date[4:-2]))
            p4= IPI(df1['FFmax'][i], p1 )
            p5= ICD(p2, p3)
            p6= IFM(p4,p5)
            if p6 >=0 and p6 < 45:
                p7= "DANGER FAIBLE"
            if p6>=45 and p6 < 60:
                p7= "DANGER MODERE"
            if p6>=60 and p6 < 90:
                p7= "DANGER SEVERE"
            if p6>=90: 
                p7= "DANGER TRES SEVERE"
                
            tab.append([stat,p1,p2,p3,p4,p5,p6,p7])
            dff = pd.DataFrame (tab, columns = ['STATION','ICL','IH','IS','IPI','ICD','IFM','CONCLUSION'])
            dff=dff.set_index('STATION')
            dff.to_excel("C:/Users/Zaineb/Desktop/METEO/PFE/OUTPUT/" + 'Indices_'+Date+'00.xlsx')
        
    except IOError:
        print("Erreur ! Le fichier n'existe pas encore")

<ipython-input-10-90b3a156c2c2>:12: RuntimeWarning: invalid value encountered in double_scalars
  fF1 = 91.9 * math.exp(-0.1386 * m) * (1.0 + pow(m, 5.31) / 49300000.0)
